In [3]:
import requests
import time
import tqdm
import pandas as pd
import os
import json
import openpyxl

In [4]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import NaN
import seaborn as sns

In [5]:
fp_data = Path.cwd() / "data" 
Path.mkdir(fp_data, exist_ok=True)  
print(fp_data)

C:\Users\dschu\OneDrive\Documenten\data


In [6]:
def log(response, logfile, timestamp):
    with open(logfile, 'a') as log_file:
        log_file.write(f"{timestamp},{response.url},{response.status_code}\n")

csv_filename = fp_data /'html_data.csv'
log_success_filename = fp_data / 'log_success.csv'
log_failure_filename = fp_data / 'log_failure.csv'

if os.path.exists(csv_filename):
    existing_data = pd.read_csv(csv_filename, header=None)
    existing_htmls = existing_data[0].tolist()
else:
    existing_htmls = []

list_htmls = existing_htmls

links = []
for page in range(1, 5):
    url = f'https://www.boliga.dk/salg/resultater?searchTab=1&sort=date-d&saleType=1&zipcodeFrom=1000&zipcodeTo=2999&salesDateMin=2017&salesDateMax=2023&page={page}&propertyType=3'
    links.append(url)

workbook = openpyxl.Workbook()
worksheet = workbook.active

for url in tqdm.tqdm(links):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP error
    except requests.exceptions.RequestException as e:
        print(url)
        print(e)
        with open(log_failure_filename, 'a') as log_fail_file:
            log_fail_file.write(f"{url},{str(e)}\n")
        continue
    
    html = response.text
    list_htmls.append(html)
    current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    log(response, log_success_filename, current_time)
    
    row_number = len(worksheet['A']) + 1
    worksheet.cell(row=row_number, column=1, value=html)

    pd.DataFrame(list_htmls).to_csv(csv_filename, index=False, header=False)

excel_filename = 'html_data.xlsx'
workbook.save(excel_filename)

print(f'HTML data saved to {excel_filename} and {csv_filename}')
print(f'Log of successful requests saved to {log_success_filename}')
print(f'Log of failed requests saved to {log_failure_filename}')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  2.00s/it]

HTML data saved to html_data.xlsx and C:\Users\dschu\OneDrive\Documenten\data\html_data.csv
Log of successful requests saved to C:\Users\dschu\OneDrive\Documenten\data\log_success.csv
Log of failed requests saved to C:\Users\dschu\OneDrive\Documenten\data\log_failure.csv


In [11]:
# Parse data with BeautifulSoup
soup = BeautifulSoup(response.content,'lxml')

# Identify table to scrape by inspecting site
table_node = soup.find('app-sold-list-table') 

In [12]:
import pandas as pd
from bs4 import BeautifulSoup

def parse_html_table(table_node):
    columns = []
    data = []
    
    # Check if the table has a thead section
    thead = table_node.find('thead')
    if thead:
        # Extract column headers
        columns_html = thead.find_all('th')
        columns = [col.text for col in columns_html]
    
    # Extract data rows
    tbody = table_node.find('tbody')
    if tbody:
        rows_list = tbody.find_all('tr')
        for row_node in rows_list:
            row = [child.text for child in row_node.children]
            data.append(row)
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

# Assuming you have 'response' already defined and 'table_node' extracted from the HTML
soup = BeautifulSoup(response.content, 'html.parser')
table_node = soup.find('table')

if table_node is not None:
    df = parse_html_table(table_node)
    print(df)
else:
    print("Table not found in the HTML content.")

                                 Boligtype & Adresse         Købesum   \
0    Ejerlejlighed EEjerlejlighed Tommerupgade 33,...   2.560.000 kr.   
1    Ejerlejlighed EEjerlejlighed Tommerupgade 25,...   2.495.000 kr.   
2    Ejerlejlighed EEjerlejlighed Tommerupgade 33,...   3.225.000 kr.   
3    Ejerlejlighed EEjerlejlighed Papirøen 63, 2. ...   9.201.410 kr.   
4    Ejerlejlighed EEjerlejlighed Tommerupgade 39,...   3.120.000 kr.   
5    Ejerlejlighed EEjerlejlighed Tommerupgade 35,...   2.700.000 kr.   
6    Ejerlejlighed EEjerlejlighed Tommerupgade 45,...   2.195.000 kr.   
7    Ejerlejlighed EEjerlejlighed Tommerupgade 45,...   2.045.000 kr.   
8    Ejerlejlighed EEjerlejlighed Papirøen 51, 5. ...   6.935.700 kr.   
9    Ejerlejlighed EEjerlejlighed Tommerupgade 25,...   1.895.000 kr.   
10   Ejerlejlighed EEjerlejlighed Tommerupgade 47,...   2.075.000 kr.   
11   Ejerlejlighed EEjerlejlighed Tommerupgade 27,...   2.445.000 kr.   
12   Ejerlejlighed EEjerlejlighed Tommerupgade 41,.

In [15]:
def parse_html_table(table_node):
    # Get the columns in a list
    columns_html = table_node.thead.find_all('th')
    # Extract the text
    columns = [col.text for col in columns_html]

    rows_list = table_node.tbody.find_all('tr')

    data = []
    for row_node in rows_list:
        row = []
        for child in row_node.children:
            row.append(child.text)
        data.append(row)
    df = pd.DataFrame(data,columns=columns)
    return df

df = parse_html_table(table_node)
df

,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,Den procentuelle forskel mellem seneste udbudspris og salgsprisen %,
0,"Ejerlejlighed EEjerlejlighed Tommerupgade 33,...",2.560.000 kr.,09-07-2023Alm. Salg,76 m² 33.684 kr/m²,2,2023,,Aktuel værdi
1,"Ejerlejlighed EEjerlejlighed Tommerupgade 25,...",2.495.000 kr.,09-07-2023Alm. Salg,87 m² 28.678 kr/m²,3,2023,,Aktuel værdi
2,"Ejerlejlighed EEjerlejlighed Tommerupgade 33,...",3.225.000 kr.,09-07-2023Alm. Salg,103 m² 31.311 kr/m²,4,2023,,Aktuel værdi
3,"Ejerlejlighed EEjerlejlighed Papirøen 63, 2. ...",9.201.410 kr.,09-07-2023Alm. Salg,122 m² 75.421 kr/m²,4,2023,,Aktuel værdi
4,"Ejerlejlighed EEjerlejlighed Tommerupgade 39,...",3.120.000 kr.,09-07-2023Alm. Salg,103 m² 30.291 kr/m²,4,2023,,Aktuel værdi
5,"Ejerlejlighed EEjerlejlighed Tommerupgade 35,...",2.700.000 kr.,09-07-2023Alm. Salg,82 m² 32.927 kr/m²,3,2023,,Aktuel værdi
6,"Ejerlejlighed EEjerlejlighed Tommerupgade 45,...",2.195.000 kr.,09-07-2023Alm. Salg,64 m² 34.297 kr/m²,2,2023,,Aktuel værdi
7,"Ejerlejlighed EEjerlejlighed Tommerupgade 45,...",2.045.000 kr.,09-07-2023Alm. Salg,67 m² 30.522 kr/m²,2,2023,,Aktuel værdi
8,"Ejerlejlighed EEjerlejlighed Papirøen 51, 5. ...",6.935.700 kr.,09-07-2023Alm. Salg,57 m² 121.679 kr/m²,2,2023,,Aktuel værdi
9,"Ejerlejlighed EEjerlejlighed Tommerupgade 25,...",1.895.000 kr.,09-07-2023Alm. Salg,63 m² 30.079 kr/m²,2,2023,,Aktuel værdi


In [16]:
# Initialize an empty DataFrame to store the parsed data
all_data = pd.DataFrame()

# Iterate over the URLs and fetch data from each page
for url in tqdm.tqdm(links):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP error
    except requests.exceptions.RequestException as e:
        print(url)
        print(e)
        with open(log_failure_filename, 'a') as log_fail_file:
            log_fail_file.write(f"{url},{str(e)}\n")
        continue

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'lxml')
    # Identify the table to scrape by inspecting the site
    table_node = soup.find('app-sold-list-table')
    
    if table_node is not None:
        # Parse the table data using the provided function
        df = parse_html_table(table_node)
        # Append the parsed data to the overall DataFrame
        all_data = all_data.append(df, ignore_index=True)
    else:
        print("Table not found in the HTML content.")

    current_time = time.strftime('%Y-%m-%d %H:%M:%S')
    log(response, log_success_filename, current_time)

# Save the DataFrame to a CSV file
all_data.to_csv(csv_filename, index=False)
print(f'HTML data saved to {csv_filename}')
print(f'Log of successful requests saved to {log_success_filename}')
print(f'Log of failed requests saved to {log_failure_filename}')

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\dschu\AppData\Local\Temp\ipykernel_23452\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df, ignore_index=True)
 25%|█████████████████████                                                               | 1/4 [00:02<00:08,  2.98s/it]C:\Users\dschu\AppData\Local\Temp\ipykernel_23452\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(df, ignore_index=True)
 50%|██████████████████████████████████████████                                          | 2/4 [00:05<00:05,  2.52s/it]C:\Users\dschu\AppData\Local\Temp\ipykernel_23452\1606127766.py:25: FutureWarning: The frame.append method is deprecated and will be remo

HTML data saved to C:\Users\dschu\OneDrive\Documenten\data\html_data.csv
Log of successful requests saved to C:\Users\dschu\OneDrive\Documenten\data\log_success.csv
Log of failed requests saved to C:\Users\dschu\OneDrive\Documenten\data\log_failure.csv


In [17]:
# Show the DataFrame in the notebook
display(all_data)

# Save the DataFrame to a CSV file
csv_filename = fp_data / 'combined_data.csv'
all_data.to_csv(csv_filename, index=False)
print(f'DataFrame saved to {csv_filename}')

,Boligtype & Adresse,Købesum,Dato & Type,m² & Kr. / m²,Vær.,Byggeår,Den procentuelle forskel mellem seneste udbudspris og salgsprisen %,
0,"Ejerlejlighed EEjerlejlighed Breslaugade 14, ...",1.401.000 kr.,19-07-2023Alm. Salg,38 m² 36.868 kr/m²,1,1976,8%,Aktuel værdi
1,Ejerlejlighed EEjerlejlighed Hans Hedtofts Ga...,5.800.000 kr.,19-07-2023Alm. Salg,112 m² 51.786 kr/m²,2,2005,-3%,Aktuel værdi
2,Ejerlejlighed EEjerlejlighed Gammel Køge Land...,1.535.000 kr.,19-07-2023Alm. Salg,50 m² 30.700 kr/m²,2,1960,-4%,Aktuel værdi
3,Ejerlejlighed EEjerlejlighed Brandholms Alle ...,1.600.000 kr.,19-07-2023Alm. Salg,58 m² 27.586 kr/m²,2,1959,-3%,Aktuel værdi
4,"Ejerlejlighed EEjerlejlighed Askevænget 25, s...",2.900.000 kr.,18-07-2023Alm. Salg,86 m² 33.721 kr/m²,4,1961,,Aktuel værdi
...,...,...,...,...,...,...,...,...
195,"Ejerlejlighed EEjerlejlighed Tommerupgade 29,...",2.425.000 kr.,09-07-2023Alm. Salg,87 m² 27.874 kr/m²,3,2023,,Aktuel værdi
196,"Ejerlejlighed EEjerlejlighed Papirøen 73, 2. ...",11.250.000 kr.,09-07-2023Alm. Salg,125 m² 90.000 kr/m²,4,2023,,Aktuel værdi
197,"Ejerlejlighed EEjerlejlighed Tommerupgade 29,...",2.375.000 kr.,09-07-2023Alm. Salg,82 m² 28.963 kr/m²,3,2023,,Aktuel værdi
198,"Ejerlejlighed EEjerlejlighed Papirøen 63, 5. ...",11.800.000 kr.,09-07-2023Alm. Salg,105 m² 112.381 kr/m²,3,2023,,Aktuel værdi


DataFrame saved to C:\Users\dschu\OneDrive\Documenten\data\combined_data.csv
